In [34]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

import yfinance as yf

from secret1 import key

Add all the imports firstly. Then lets import our stocks using a csv file!

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]


Lets set up and query the API.

In [3]:
symbol = 'AAPL'
stock = yf.Ticker(symbol)

Lets put this in a pandas dataframe to use.

In [4]:
columns = ["Ticker","Price","MarketCap","NoShares"]
df1 = pd.DataFrame(columns=columns)
df1

,Ticker,Price,MarketCap,NoShares


Lets add our stock price and market cap.

In [5]:
# df = df.append(pd.Series(["AAPL",stock.info['currentPrice'],stock.info['marketCap'],"N/A"],index=columns),ignore_index=True)

lst = []
for item in stocks['Ticker']:
    obj = yf.Ticker(item)
    row = {"Stock":item,"Price":obj.info.get("currentPrice"),"MarketCap":obj.info.get("marketCap"),"NoShares":"N/A"}
    lst.append(row)

df = pd.DataFrame(lst)
print(df)

#df_combined = pd.concat([df,df_new],ignore_index=True)
#df_combined



    Stock   Price     MarketCap NoShares
0       A  113.23  3.263617e+10      N/A
1     AAL   11.74  7.746392e+09      N/A
2     AAP   60.43  3.621304e+09      N/A
3    AAPL  213.25  3.164715e+12      N/A
4    ABBV  196.30  3.467757e+11      N/A
..    ...     ...           ...      ...
500   YUM  141.34  3.928757e+10      N/A
501   ZBH   91.15  1.803385e+10      N/A
502  ZBRA  308.31  1.567608e+10      N/A
503  ZION   51.52  7.744743e+09      N/A
504   ZTS  146.50  6.522297e+10      N/A

[505 rows x 4 columns]


Remember that to improve performance we can use batch api calls as http requests are the slowest components of a script. This basically gets us a large batch all at once instead of doing it 505 times. Pagination is used to again split the data into separate pages for easier access. Hence 

In [6]:
newlst = []
def chunks(arr,n):
    for i in range(0,len(arr),n):
        newlst.append(arr[i:i+n])

chunks(lst,100)

for item in newlst:
    print(item)
    print("\n")

[{'Stock': 'A', 'Price': 113.23, 'MarketCap': 32636170240, 'NoShares': 'N/A'}, {'Stock': 'AAL', 'Price': 11.74, 'MarketCap': 7746392064, 'NoShares': 'N/A'}, {'Stock': 'AAP', 'Price': 60.43, 'MarketCap': 3621304064, 'NoShares': 'N/A'}, {'Stock': 'AAPL', 'Price': 213.25, 'MarketCap': 3164715352064, 'NoShares': 'N/A'}, {'Stock': 'ABBV', 'Price': 196.3, 'MarketCap': 346775748608, 'NoShares': 'N/A'}, {'Stock': 'ABC', 'Price': None, 'MarketCap': None, 'NoShares': 'N/A'}, {'Stock': 'ABMD', 'Price': None, 'MarketCap': None, 'NoShares': 'N/A'}, {'Stock': 'ABT', 'Price': 130.93, 'MarketCap': 227878420480, 'NoShares': 'N/A'}, {'Stock': 'ACN', 'Price': 247.54, 'MarketCap': 154181025792, 'NoShares': 'N/A'}, {'Stock': 'ADBE', 'Price': 345.62, 'MarketCap': 146611994624, 'NoShares': 'N/A'}, {'Stock': 'ADI', 'Price': 220.69, 'MarketCap': 109516972032, 'NoShares': 'N/A'}, {'Stock': 'ADM', 'Price': 56.73, 'MarketCap': 27683502080, 'NoShares': 'N/A'}, {'Stock': 'ADP', 'Price': 301.64, 'MarketCap': 1222546

In [35]:
api_url = f"https://finnhub.io/api/v1/quote?symbol=AAPL&token={key}"
stck = requests.get(api_url).json()
print(stck)

{'c': 213.25, 'd': 10.33, 'dp': 5.0907, 'h': 215.38, 'l': 205.59, 'o': 205.63, 'pc': 202.92, 't': 1754510400}


Calculating the number of shares to buy.

In [8]:
portfolio_size = input("Size of portfolio")

try:
    value = float(portfolio_size)

except ValueError:
    portfolio_size = input("Size of portfolio")
    value = float(portfolio_size)

Every stock needs to have the same position size as it is equal weight.

In [9]:
position_size = value/len(df.index) # df.index gives a range and len of that is no. of rows.
print(df.loc[0,"Price"])
for i in range(0,len(df.index)):
    price = df.loc[i,"Price"]
    if pd.notna(price) and price > 0:
        df.loc[i,"NoShares"] = math.floor(position_size/df.loc[i,"Price"])
    

df

113.23


,Stock,Price,MarketCap,NoShares
0,A,113.23,3.263617e+10,17
1,AAL,11.74,7.746392e+09,168
2,AAP,60.43,3.621304e+09,32
3,AAPL,213.25,3.164715e+12,9
4,ABBV,196.30,3.467757e+11,10
...,...,...,...,...
500,YUM,141.34,3.928757e+10,14
501,ZBH,91.15,1.803385e+10,21
502,ZBRA,308.31,1.567608e+10,6
503,ZION,51.52,7.744743e+09,38


Lets format our output using XlsxWriter.

In [30]:
writer = pd.ExcelWriter("trades.xlsx",engine="xlsxwriter")
df.to_excel(writer,sheet_name="Trades",index=False)


Add formatting to it now. These include colors fonts symbols.
We need 4 main formats:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase




In [31]:
bg_col = "#0a0a23"
font_col = "#ffffff"

header_format = writer.book.add_format(

    {
        "font_color": bg_col,
        "bg_color" : font_col,
        "border" : 2
    }
)

string_format = writer.book.add_format(

    {
        "font_color": font_col,
        "bg_color": bg_col,
        "border": 1
    }


)

dollar_format = writer.book.add_format(

    {
        "num_format": "$0.00",
        "font_color": font_col,
        "bg_color": bg_col,
        "border": 1
    }


)

integer_format = writer.book.add_format(

    {
        "num_format": "0",
        "font_color": font_col,
        "bg_color": bg_col,
        "border": 1
    }


)


After creating these formats lets apply them using the set_colummn method.
```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [18]:
writer.sheets["Trades"].set_column("A:A",18,string_format)
writer.close()

But instead of doing one at a time lets just use a for loop.

In [32]:
column_formats = {

    "A": ["Stock",string_format],
    "B": ["Price",dollar_format],
    "C": ["Market Cap",dollar_format],
    "D": ["No.Shares",integer_format]

}

for key in column_formats.keys():
    writer.sheets["Trades"].write(f"{key}1", column_formats[key][0],header_format)
    writer.sheets["Trades"].set_column(f"{key}:{key}",20,column_formats[key][1])

writer.close()